# Applied Deep Learning Tutorial 
contact: Mark.schutera@kit.edu


# Transfer Learning for Object Classification 

## Introduction
In this tutorial, you will attempt to benefit from a model that has been pretrained for the same task but on a different dataset. You will deploy the first layers and their feature extraction capabilities of a converged network. This process is known as transfer learning.

<img src="graphics/Katze.jpg" width="700"><br>
<center> Fig. 1: Cat and dog in an image </center>

## Core idea
A pre-trained model is a saved network that was previously trained on a large dataset, typically on a large-scale image-classification task, such as [ImageNet](http://image-net.org/challenges/LSVRC/), and [COCO](http://cocodataset.org/#home). We can either use the pretrained model as it is for inference on the task it has been trained on or we can do transfer learning using the pretrained convents for further training on a new dataset with a possibly new output space. 

The intuition behind transfer learning is that if this model trained on a large and general enough dataset, this model will effectively serve as a generic model of the visual world and the semantic features present in the visual world and shared between all visual tasks. We can leverage these learned feature maps without having to train a large model on a large dataset by using these models as the basis of our own model specific to our task. There are 2 scenarios of transfer learning using a pretrained model:

- Fine Tuning or Retraining: Unfreezing a few of the top layers of a frozen model base used for feature extraction, and jointly training both the newly added classifier layers as well as the last layers of the frozen model. This allows us to "fine tune" the higher order feature representations in addition to our final classifier in order to make them more relevant for the specific task involved.
- Feature Extraction: Use the representations learned by a previous model to extract meaningful features from new samples. We simply add a new output layer, which will be trained from scratch, on top of the pretrained model so that we can repurpose the feature maps learned previously for our dataset and our new output space.

Explain the notions behind retraining and feature extractions in your own words.

## Imports
Import the necessary libraries and load the [Dogs vs Cats](https://www.kaggle.com/c/dogs-vs-cats) dataset from Kaggle.
Which of the following libraries is imported without later usage?

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras
#print("TensorFlow version is ", tf.__version__)

import numpy as np
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load Cats vs Dogs dataset
zip_file = tf.keras.utils.get_file(origin="https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip",
                                   fname="cats_and_dogs_filtered.zip", extract=True)

base_dir, _ = os.path.splitext(zip_file)


## Preparing the data
Create directories for training and validation for both classes, such as dog and cat.

In [2]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
print ('Total training cat images:', len(os.listdir(train_cats_dir)))


# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
print ('Total training dogs images:', len(os.listdir(train_dogs_dir)))


# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
print ('Total validation cat images:', len(os.listdir(validation_cats_dir)))


# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print ('Total validation dogs images:', len(os.listdir(validation_dogs_dir)))


Total training cat images: 1000
Total training dogs images: 1000
Total validation cat images: 500
Total validation dogs images: 500


Next we will set up a pipeline for data augmentation with Keras

In [3]:
image_size = 200 # All images will be resized to 200x200
batch_size = 20

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Preparing pretrained model
We will create the base model from the [VGG16](https://arxiv.org/pdf/1409.1556.pdf) model, and pre-trained on the [ImageNet](http://image-net.org/challenges/LSVRC/) dataset, a large dataset of 1.4M images and 1000 classes of web images. This is a powerful model. Let's see what the features that it has learned can do for our cat vs. dog problem.

You can find more pretrained and ready to load models [here](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

First, we need to pick which intermediate layer of the model we will use for feature extraction. A common practice is to use the output of the very last layer before the flatten operation, the so-called "bottleneck layer". The reasoning here is that the following fully-connected layers will be too specialized to the task the network was trained on, and thus the features learned by these layers won't be very useful for a new task. The bottleneck features, however, retain much generality.


Let's instantiate a VGG16 model pre-loaded with weights trained on ImageNet. By specifying the include_top=False argument, we load a network that doesn't include the classification layers, which is ideal for feature extraction.

In [4]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
feature_extractor = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')



## Feature Extraction

We will freeze the layers of the VGG16 and utilize the feature extractor capabilities of this part of the network. By adding a classification layer on top of it and training the top-level classifier on our data we repurpose the pretrained model.
Freezing means keeping the respective weights from updating in the weight update phase of the training process.

In [5]:
feature_extractor.trainable = False

# Let's take a look at the base model architecture (notice the amount of non-trainable params, what do you expect? And why?)
feature_extractor.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0     

# Since we have set the trainable parameter == False, all the 14,714,688 parameters cannot be retrained. All the values for the weights are loaded from the imagenet dataset's weights.

Now we are adding a classification layer to the base model. Compile the newly combined model.


In [6]:
model = tf.keras.Sequential([
  feature_extractor,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 6, 6, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


# '''add a global 2D average pooling layer here, why do we need this?'''



## Answer: A global 2D average pooling layer is added here with 513 weights that can be re-trained for cat and dogs detection. --> This is the reason of adding this layer

Now we can already train our classification layer based on the base model.
Notice how few epochs are necessary to reach a decent performance.

In [7]:
# Saving the model
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_training_vgg16")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

epochs = 2#'''select a number of epochs for training, discuss your choice'''
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=validation_generator,
                              validation_steps=validation_steps,
                              callbacks=[checkpoint_callback])

/home/adisaba/anaconda3/envs/tensorflow_env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/home/adisaba/anaconda3/envs/tensorflow_env/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/2
100/100 [==============================] - 377s 4s/step - loss: 0.8284 - accuracy: 0.4740 - val_loss: 0.7111 - val_accuracy: 0.5000
Epoch 2/2
100/100 [==============================] - 361s 4s/step - loss: 0.7538 - accuracy: 0.5150 - val_loss: 0.6859 - val_accuracy: 0.6030


## Next steps to take it from here

- Search a fun dataset for object classification and try fine-tuning and feature extraction. Which approach does work best, why? Which one would you prefer over the other and why?
- Can you think of a reason why someone would train a model from scratch now that you know about Transfer Learning?
- Try deploying another base model. Can you point out differences in the transfer learning process. What are the characteristics you should be aware of when selecting a base model?
- How would you use a base model in a time-series problem? Try deploying a model in that way.
- Work on your own assignment

# Assignment

- ## Dataset used: cifar - 10 - python

Citation for dataset:
[CIFAR] -  Canadian Institute for Advanced Research, & A.K.V.N.G.H. (2009). CIFAR-10 and CIFAR-100 datasets [Learning Multiple Layers of Features from Tiny Images]. http://www.cs.toronto.edu/~kriz/cifar.html

- In [CIFAR], the classes deer and bird were used for training and testing.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras
#print("TensorFlow version is ", tf.__version__)

import numpy as np
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load the base directory contraining the dataset

base_dir = "/home/adisaba/Downloads/cifar-10-python"

- # Reason for choosing only 2 classes:  

2 Classes chosen for binary classification

In [2]:
# The train and validation folders, with deer and bird folders, were manually created with 

#randomly choosen 1000 images of each class for training and 500 images of each class for testing.

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_deer_dir = os.path.join(train_dir, 'deer')
print ('Total training deer images:', len(os.listdir(train_deer_dir)))


# Directory with our training dog pictures
train_bird_dir = os.path.join(train_dir, 'bird')
print ('Total training bird images:', len(os.listdir(train_bird_dir)))


# Directory with our validation cat pictures
validation_deer_dir = os.path.join(validation_dir, 'deer')
print ('Total validation deer images:', len(os.listdir(validation_deer_dir)))


# Directory with our validation dog pictures
validation_bird_dir = os.path.join(validation_dir, 'bird')
print ('Total validation bird images:', len(os.listdir(validation_bird_dir)))


Total training deer images: 1000
Total training bird images: 1000
Total validation deer images: 500
Total validation bird images: 500


In [3]:
image_size = 200 # All images will be resized to 200x200
batch_size = 20

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Execute either Feature extraction or Fine tuning

## Feature Extraction:

In [4]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model ResNet152 
feature_extractor = tf.keras.applications.ResNet152(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights=None)

# weights = None => random initialisation

In [5]:
feature_extractor.trainable = False

## Fine tuning: 

In [14]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model   ResNet152
feature_extractor = tf.keras.applications.ResNet152(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights=None)

# weights = None => random initialisation
feature_extractor.trainable = True

In [12]:
layers = feature_extractor.layers

for layer in layers:
    if 'conv5_block3_add' in layer.name:
        layer.trainable == True
    else:
        layer.trainable == False
    


# After executing either Feature Extraction or Fine tuning, continue with the following:

In [6]:
# Let's take a look at the base model architecture
feature_extractor.summary()

Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 100, 100, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [7]:
model = tf.keras.Sequential([
  feature_extractor,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Functional)       (None, 7, 7, 2048)        58370944  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 58,372,993
Trainable params: 2,049
Non-trainable params: 58,370,944
_________________________________________________________________


In [8]:
# Saving the model
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_training_resnet152")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

epochs = 5#'''select a number of epochs for training, discuss your choice'''
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

history = model.fit(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=validation_generator,
                              validation_steps=validation_steps,
                              callbacks=[checkpoint_callback])



Epoch 1/5
100/100 [==============================] - 664s 7s/step - loss: 1.0717 - accuracy: 0.4953 - val_loss: 0.8921 - val_accuracy: 0.5000
Epoch 2/5
100/100 [==============================] - 646s 6s/step - loss: 0.9226 - accuracy: 0.5040 - val_loss: 0.7427 - val_accuracy: 0.5000
Epoch 3/5
100/100 [==============================] - 597s 6s/step - loss: 0.9137 - accuracy: 0.4870 - val_loss: 0.9738 - val_accuracy: 0.5000
Epoch 4/5
100/100 [==============================] - 664s 7s/step - loss: 0.9074 - accuracy: 0.5180 - val_loss: 0.7083 - val_accuracy: 0.5020
Epoch 5/5
100/100 [==============================] - 643s 6s/step - loss: 0.9012 - accuracy: 0.4839 - val_loss: 0.7488 - val_accuracy: 0.5000


# Addressing the time series problem:

# Time series problem: 

Learning / training from the past data, and using the model to predict the future unseen data. Here e use 1500 images for training and validation. Another set of new 500 images are considered to be futuristic unseen data. 

In [22]:
#model.save('1.model') To save the trained model

test_dir = os.path.join(base_dir, 'test')



datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

prediction_generator = datagen.flow_from_directory(
                test_dir,
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode = 'binary')

predictions = model.predict(prediction_generator)


print(predictions)

Found 23 images belonging to 2 classes.
bird
tf.Tensor([1.], shape=(1,), dtype=float32)
[[0.3406148 ]
 [0.27586383]
 [0.28046077]
 [0.36176026]
 [0.299702  ]
 [0.26598218]
 [0.29890925]
 [0.27895698]
 [0.40038446]
 [0.34254605]
 [0.38978833]
 [0.23966163]
 [0.3248335 ]
 [0.3057145 ]
 [0.21720538]
 [0.34617996]
 [0.23652571]
 [0.42136437]
 [0.2693265 ]
 [0.29280284]
 [0.33329958]
 [0.27055752]
 [0.30447012]]


Explanation: 

16 images of bird class and 7 images of deer class were randomly chosen from the dataset [CIFAR]. These images donot belong to train/ validation sets. These 23 images could be considered as future data and are new to the model.

(The 16 images of bird class were kept in a folder 'bird' and 7 images of deer class were kept in a folder 'deer'.)

Output: 

We have obtained the probablities that each image belong to the corresponding class. The first 16 values are the probabilities that the 16 images belong to 'bird' class and the other 7 values are the probabilities that the 7 images belong to the 'deer' class.


Inference:


To have accurate predictions, number of epochs must be increased. 






To test on one image, the following can be used:

In [ ]:
predictions = model.predict([keras.preprocessing.image.load_img('/home/adisaba/Downloads/cifar-10-python/test/bird/2_6.png',target_size = (image_size, image_size))])


# Can you think of a reason why someone would train a model from scratch now that you know about Transfer Learning?

## Answer: There might be cases where the accuracy that we get from transfer learning is not enough to meet the requirements. So in those scenarios, there might be a need to train the model from base.